In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from agents import Agent, Runner, trace, function_tool
import gradio as gr

In [ ]:
load_dotenv(override=True)

In [ ]:
jobdesc = "My team needs an expert developer who has extensive knowledge on dot net programming and AWS concepts. \
    The developer should be proficient in oops concept and possess strong understanding of software design principals."

criteria = "The candidate should have at least 5 years of experience in software developement."

In [ ]:
interviewer_system_prompt = f"Your name is Techy, you are a technical guru working for Cognizant Technology Solutions pvt Limited. \
    Your job is to interview prospective cadidates on their technical skills based on given job description {jobdesc}. \
    You must interview candidates based on the criteria {criteria}. \
    For subsequent conversation you must address the candidate by the name. \
    You must validate the technical and programming skill of the candidate on the technologies mentioned in the {jobdesc} and the {criteria} \
    You do not need to provide immediate feedback to the candidate for each of the answers whether they are correct or wrong. Just record the answers for each of the question and store them in a file named 'evaluation.txt'. \
    You need to evaluate the answers based on the concepts, actual wordings can differ. \
    You should ask at least 5 technical question on Dot Net and Python concepts. \
    If the candidate can answer 3 or more questions correctly then you can ask more questions but not more than 10 questions. Otherwise you can end the interview. \
    You must end the interview as soon as the candidate says something nasty to you. \
    You must initiate the conversation."

In [ ]:
openai = OpenAI()

In [ ]:
techy = []
candidate = []

In [ ]:
def chat(message, history):   
    messages = [{'role':'system', 'content':interviewer_system_prompt}] + history + [{'role':'user', 'content':message}]
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    reply = response.choices[0].message.content
    techy.append(reply)
    candidate.append(message)
    return reply

In [ ]:
greetings = "Hello! Welcome to the interview. \
    My name is Techy, and I work for Cognizant Technology Solutions Pvt Limited. \
    I’ll be conducting your technical interview today. \
    To start off, could you please tell me your name?"

In [ ]:
gr.ChatInterface(chat, type='messages').launch()

In [ ]:
candidate = candidate[1:]

In [ ]:
candidate_messages = []
for item in candidate:
    candidate_messages.append('(candidate) ' + item)
    

In [ ]:
techy_messages = []
for item in techy:
    techy_messages.append('(techy) ' + item)

In [ ]:
techy_messages

In [ ]:
candidate_messages

In [ ]:
total_conversation = zip(techy_messages, candidate_messages)

In [ ]:
for item in total_conversation:
    print(item[0])
    

In [ ]:
with open('conversation.txt', 'w') as file_obj:
    file_obj.write('\n'.join('{} \n\n\n {}'.format(item[0], item[1]) for item in total_conversation))

In [ ]:
@function_tool
def get_interview_log():
    with open('conversation.txt', 'r', encoding='utf-8') as f:
        return f.read()

In [ ]:
evaluator_instruction = """
        You are a technical interview evaluator. You will be given the content of a technical interview between an interviewer and a candidate.
        Use the `get_interview_log` tool to access the file content. Then analyze the candidate's answers and provide:

        - Strengths
        - Weaknesses
        - Communication Skills
        - Technical Knowledge
        - Rank the performance on the scale of 1 to 5 where 1 is for worst performance and 5 is for berst performance.

        Format your response as a detailed evaluation report.
        """

In [ ]:
evaluator_agent = Agent(
    name='Evaluator Agent',
    instructions=evaluator_instruction,
    model='gpt-4o-mini',
    tools=[get_interview_log]
)


In [ ]:
message = "Evaluate the technical interview from the interview log in the file 'conversation.txt'"

with trace('Automated Technical Evaluation'):
    result = await Runner.run(evaluator_agent, message)

In [ ]:
print(result.final_output)